In [1]:
import requests
import wikipediaapi

# Replace 'YOUR_API_KEY' with your TMDb API key
API_KEY = 'e1b016085c92ba0d096ab336260d1b6a'

# Initialize variables
top_movies = []
page_number = 1
num_movies = 10000

# Continue fetching pages until we have the top 100 movies
while len(top_movies) < num_movies:
    # Define the TMDb API endpoint to get the top-rated movies for a specific page
    url = f"https://api.themoviedb.org/3/movie/top_rated?api_key={API_KEY}&language=en-US&page={page_number}"

    # Send a GET request to retrieve the current page of top-rated movies
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        current_page_movies = data.get("results", [])

        # Add the current page of movies to the list
        top_movies.extend(current_page_movies)

        # Increment the page number for the next request
        page_number += 1
    else:
        print(f"Failed to retrieve data from the TMDb API for page {page_number}.")
        break

wiki_wiki = wikipediaapi.Wikipedia('MovieRex (joeljoy1912@gmail.com)', "en")


def get_plot_summary(movie_title):
    page = wiki_wiki.page(movie_title)
    if page.exists():
        return page.summary
    else:
        return "Plot summary not found on Wikipedia."
    
movies = []
# Print the top 100 movie titles and their plots
for i, movie in enumerate(top_movies[:num_movies], start=1):
    title = movie.get("title", "Title not found")
    overview = movie.get("overview", "Overview not available")
    #wiki_summary = get_plot_summary(title)
    movies.append((title,overview))
    print(f"{i}. Title: {title}")
    print(f"   Plot: {overview}\n")


1. Title: The Godfather
   Plot: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American Corleone crime family. When organized crime family patriarch, Vito Corleone barely survives an attempt on his life, his youngest son, Michael steps in to take care of the would-be killers, launching a campaign of bloody revenge.

2. Title: The Shawshank Redemption
   Plot: Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.

3. Title: The Godfather Part II
   Plot: In the continuing saga of the Corleone crime family, a young Vito Corleone grows up in Sicily and in 1910s New York. In the 1950s, Michael Corleone attempts to expand the family busin

In [1]:
import pandas as pd
import string

df = pd.DataFrame(movies)
df.rename(columns={0: 'title', 1: 'imdb_summary'}, inplace=True)
df.to_csv('imdb_data.csv', encoding='utf-8')


NameError: name 'movies' is not defined

In [3]:
import numpy as np
import pandas as pd
import os.path

df = pd.read_csv('c:/Users/joelj/OneDrive/Documents/classes/Independent Study/imdb_data.csv')
df.head()

,Unnamed: 0,title,imdb_summary
0,0,The Godfather,"Spanning the years 1945 to 1955, a chronicle o..."
1,1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...
2,2,The Godfather Part II,In the continuing saga of the Corleone crime f...
3,3,Schindler's List,The true story of how businessman Oskar Schind...
4,4,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second..."


In [4]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joelj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\joelj\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\joelj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joelj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df['imdb_summary'] = df['imdb_summary'].astype(str)

In [6]:
def preprocess_sentences(text):
    #text = text.lower()
    temp_sent =[]
    words = nltk.word_tokenize(text)
    tags = nltk.pos_tag(words)
    for i, word in enumerate(words):
        if tags[i][1] in VERB_CODES:
            lemmatized = lemmatizer.lemmatize(word, 'v')
        else:
            lemmatized = lemmatizer.lemmatize(word)
        if lemmatized not in stop_words and lemmatized.isalpha():
            temp_sent.append(lemmatized)
            
    finalsent = ' '.join(temp_sent)
    finalsent = finalsent.replace("n't", " not")
    finalsent = finalsent.replace("'m", " am")
    finalsent = finalsent.replace("'s", " is")
    finalsent = finalsent.replace("'re", " are")
    finalsent = finalsent.replace("'ll", " will")
    finalsent = finalsent.replace("'ve", " have")
    finalsent = finalsent.replace("'d", " would")
    return finalsent

df["processed"]= df["imdb_summary"].apply(preprocess_sentences)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

tfidfvec = TfidfVectorizer()
tfidf_movieid = tfidfvec.fit_transform((df["processed"]))

sparse_matrix = csr_matrix(tfidf_movieid)
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(sparse_matrix, sparse_matrix)
cos_sim.shape


(10000, 10000)

In [8]:

def recommendations(title, cosine_sim = cos_sim):
	recommended_movies = []
	index = df.loc[df['title'] == title].index[0]
	similarity_scores = pd.Series(cosine_sim[index]).sort_values(ascending = False)
	top_10_movies = list(similarity_scores.iloc[1:11].index)
	for i in top_10_movies:
		recommended_movies.append(list(df['title'])[i])
	return recommended_movies


In [15]:
recommendations("Toy Story")

['Toy Story 3',
 'Toy Story 2',
 'Toy Story 4',
 'The 40 Year Old Virgin',
 'Small Fry',
 "Child's Play 2",
 'Buzz Lightyear of Star Command: The Adventure Begins',
 'Lightyear',
 'Man on the Moon',
 'Woody Woodpecker']

In [2]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

C:\Users\joelj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\joelj\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [16]:
import torch
def encode(document: str) -> torch.Tensor:
    tokens = tokenizer(document, return_tensors='pt')
    vector = model(**tokens)[0].detach().squeeze()
    return torch.mean(vector, dim=0)